import lib

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import math

load dataset

In [96]:
data = load_breast_cancer()
data

{'DESCR': '.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 569\n\n    :Number of Attributes: 30 numeric, predictive attributes and the class\n\n    :Attribute Information:\n        - radius (mean of distances from center to points on the perimeter)\n        - texture (standard deviation of gray-scale values)\n        - perimeter\n        - area\n        - smoothness (local variation in radius lengths)\n        - compactness (perimeter^2 / area - 1.0)\n        - concavity (severity of concave portions of the contour)\n        - concave points (number of concave portions of the contour)\n        - symmetry \n        - fractal dimension ("coastline approximation" - 1)\n\n        The mean, standard error, and "worst" or largest (mean of the three\n        largest values) of these features were computed for each image,\n        resulting in 30 features.  For

convert dataset in dataframe

In [0]:
df = pd.DataFrame(data['data'], columns = data['feature_names'], index=None)
df['target'] = data['target']

show dataframe

In [8]:
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


train test split

In [0]:
y = np.array(df['target'])
X = np.array(df.iloc[:, :-1])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

*Decision* tree sklearn's

In [99]:
model = DecisionTreeClassifier(random_state=1)
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
print('acc', acc)

acc 0.951048951048951


Decision tree with 1 node

In [100]:
model1 = DecisionTreeClassifier(random_state=1, max_depth=1)
model1.fit(X_train, y_train)
acc = model1.score(X_test, y_test)
print('acc', acc)

acc 0.8951048951048951


function for Custom tree (IG 3)

In [0]:
def log_funct(x):
  if x == 0:
    return 0
  else:
    return math.log(x)

In [0]:
def entropy(p_pos, p_neg):
  p_pos_l = log_funct(p_pos)
  p_neg_l = log_funct(p_neg)
  return -(p_pos * p_pos_l + p_neg * p_neg_l)

In [103]:
def entropy_count(y):
  sum_num = len(y)
  if sum_num == 0:
    return 0
  else:
    p_pos = sum(y) / sum_num
    p_neg = 1 - p_pos
    return entropy(p_pos, p_neg)

entropy_count(y_train)

0.6594287218675621

In [104]:
def entropy_loss(X, y, num_column, pred):
  y_b, y_s = [], []
  for index in range(X.shape[1]):
    if X[index][num_column] > pred:
      y_b.append(y[index])
    else:
      y_s.append(y[index])
  entropy_b = entropy_count(y_b)
  entropy_s = entropy_count(y_s)
  num_b = len(y_b)
  num_s = len(y_s)
  entropy_0 = entropy_count(y)
  loss = entropy_0 - (num_b * entropy_b + num_s * entropy_s) / (num_b + num_s)
  return loss, y_b, y_s

loss = entropy_loss(X_train, y_train, 0 , X[0][0])
print('loss', loss)

loss (0.18269314271567305, [0, 0, 0], [1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1])


In [0]:
def prune(X, y, num_column, pred):
  loss, y_b, y_s = entropy_loss(X, y, num_column, pred)
  decision  = sum(y_b) >= sum(y_s)
  return decision


Class Custome tree

In [0]:
class Customtree():
  def __init__(self):
    self.node = []
    self.decision = []
  def fit(self, X_train, y_train):
    column_score = [[],[]]
    for column in range(0, X_train.shape[1]):
      loss_elem = []
      for element in X_train[column]:
        loss_elem.append(entropy_loss(X_train, y_train, column, element)[0])
      best_elem = np.argmax(loss_elem)
      best_loss = np.max(loss_elem)
      column_score[0].append(best_loss)
      column_score[1].append(best_elem)
    best = np.argmax(column_score, axis=1)[0]
    self.node.append((best, column_score[1][best]))
    self.decision.append(prune(X_train, y_train, best, column_score[1][best]))
  def score(self, X_test, y_test):
    y_pred = np.zeros_like(y_test)
    for index in range(len(y_test)):
      compare_res = X_test[index][self.node[0][0]] > self.node[0][1]
      y_pred[index] = int(compare_res == self.decision)
    score = sum(y_test == y_pred) / len(y_test)
    return score

In [107]:
column_score = [[],[]]
for column in range(0, X.shape[1]):
  loss_elem = []
  for element in X_train[column]:
    loss_elem.append(entropy_loss(X, y, column, element)[0])
  best_elem = np.argmax(loss_elem)
  best_loss = np.max(loss_elem)
  column_score[0].append(best_loss)
  column_score[1].append(best_elem)
best = np.argmax(column_score, axis=1)[0]
print(column_score)
print(best)

[[0.5009711247651174, 0.4693620987067837, 0.4566949151769297, 0.37323492639334127, 0.3634535282190177, 0.4262278874933878, 0.5853383299127197, 0.5853383299127197, 0.3634535282190177, 0.3682244657380191, 0.5481477380270181, 0.38408020978450946, 0.4177267049828618, 0.4693620987067837, 0.36739363210752907, 0.4262278874933878, 0.3944493501369233, 0.4177267049828618, 0.3634535282190177, 0.4454670558172589, 0.41784575760098697, 0.39625010358397006, 0.4177267049828618, 0.4177267049828618, 0.38408020978450946, 0.43538229134770423, 0.4566949151769297, 0.5481477380270181, 0.3590180857226804, 0.41784575760098697], [20, 20, 22, 3, 24, 24, 5, 6, 28, 29, 10, 12, 11, 21, 14, 7, 18, 15, 7, 19, 1, 1, 2, 3, 26, 25, 26, 5, 10, 27]]
6


In [108]:
model = Customtree()
model.fit(X_train, y_train)
print(model.node)
print(model.decision)
model.score(X_test, y_test)

[(2, 22)]
[True]


0.6223776223776224

In [109]:
test = pd.DataFrame({'1':[1, 2, 4], '2':[3, 6, 7]})
#print(test)
test[test['1']>1]
np.shape(X)

(569, 30)

bagging

In [110]:
X_bag = np.concatenate((X_train, y_train.reshape([426, 1])), axis=1)
X_bag[1]

array([1.340e+01, 2.052e+01, 8.864e+01, 5.567e+02, 1.106e-01, 1.469e-01,
       1.445e-01, 8.172e-02, 2.116e-01, 7.325e-02, 3.906e-01, 9.306e-01,
       3.093e+00, 3.367e+01, 5.414e-03, 2.265e-02, 3.452e-02, 1.334e-02,
       1.705e-02, 4.005e-03, 1.641e+01, 2.966e+01, 1.133e+02, 8.444e+02,
       1.574e-01, 3.856e-01, 5.106e-01, 2.051e-01, 3.585e-01, 1.109e-01,
       0.000e+00])

In [111]:
np.random.shuffle(X_bag)
X_bag[0:2,-1].shape

(2,)

In [0]:
def bag(num_trees, X, bag_model, percent=0.6):
  list_model = []
  for tree in range(num_trees):
    num_row = int(len(X)*percent)
    np.random.shuffle(X)
    X_train = X[0:num_row,:-1]
    y_train = X[0:num_row,-1]
    if bag_model == 'sklearn':
      model = DecisionTreeClassifier(random_state=1, max_depth=1)
    else:# bag_model == 'custom':
      model = Customtree()
    model.fit(X_train, y_train)
    list_model.append(model)
  return list_model

Bagging  for custome tree

In [113]:
for num_tree in [5, 10]:
  list_trees = bag(num_tree, X_bag, 'custome')
  list_acc = []
  for tree in list_trees:
    acc = tree.score(X_test, y_test)
    list_acc.append(acc)
  print('num_trees:', num_tree)
  print('accuracy:', list_acc)
  print('mean_acc:', np.mean(list_acc), '\n')

num_trees: 5
accuracy: [0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224]
mean_acc: 0.6223776223776224 

num_trees: 10
accuracy: [0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.6223776223776224, 0.8601398601398601, 0.6223776223776224, 0.8601398601398601, 0.6223776223776224]
mean_acc: 0.6699300699300699 



Bagging for sklearn tree

In [114]:
for num_tree in [5, 10]:
  list_trees = bag(num_tree, X_bag, 'sklearn')
  list_acc = []
  for tree in list_trees:
    acc = tree.score(X_test, y_test)
    list_acc.append(acc)
  print('num_trees:', num_tree)
  print('accuracy:', list_acc)
  print('mean_acc:', np.mean(list_acc), '\n')

num_trees: 5
accuracy: [0.8951048951048951, 0.951048951048951, 0.8951048951048951, 0.8951048951048951, 0.916083916083916]
mean_acc: 0.9104895104895105 

num_trees: 10
accuracy: [0.916083916083916, 0.951048951048951, 0.916083916083916, 0.8951048951048951, 0.916083916083916, 0.951048951048951, 0.951048951048951, 0.8881118881118881, 0.951048951048951, 0.8951048951048951]
mean_acc: 0.9230769230769231 



In [115]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator = DecisionTreeClassifier(max_depth=1, random_state=1) ,n_estimators=5, warm_start=True, random_state=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9230769230769231

In [0]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator = Customtree() ,n_estimators=5, warm_start=True, random_state=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)